In [110]:
!pip install tqdm

In [111]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from IPython import display
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.image import resize, resize_with_pad
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tqdm.notebook import tqdm

In [3]:
# Load the document file into memory
def load_doc(filename):
    # Open file to read
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
!pwd

/Users/ChrisKarg/code/CMaxK/robo_romeo/notebooks


In [5]:
dataframe = load_doc("../raw_data/descriptions.txt")

In [6]:
dataframe = dataframe.split('\n')

In [7]:
len(dataframe)

40460

In [8]:
dataframe[:2]

['1000268201_693b08cb0e a child in a pink dress is climbing up a set of stairs in an entry way',
 '1000268201_693b08cb0e a girl going into a wooden building']

In [9]:
l_all =[]
for n in dataframe:
    line = {"id": n.split(" ")[0],
        'value': "startsequence " + " ".join(n.split(" ")[1:]) + " endsequence"}
    l_all.append(line)
    

In [10]:
import pandas as pd
df_all = pd.DataFrame(l_all)

/Users/ChrisKarg/.pyenv/versions/3.8.13/envs/lewagon/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
t = Tokenizer()
t.fit_on_texts(df_all.value)

In [13]:
df_all["value_tokenized"] = t.texts_to_sequences(df_all.value)

In [127]:
train_ids = np.unique(df_all.id)[:int(0.8*len(np.unique(df_all.id)))]
test_ids = np.unique(df_all.id)[int(0.8*len(np.unique(df_all.id))):]

In [129]:
len(test_ids)

1619

In [131]:
df_train = df_all[df_all.id.isin(train_ids)]
df_test = df_all[df_all.id.isin(test_ids)]

In [45]:
final_array = np.array(list(imgs_to_load.ravel()))

In [47]:
final_array.shape

(16,)

In [48]:
final_array

array(['1000268201_693b08cb0e', '1000268201_693b08cb0e',
       '1000268201_693b08cb0e', '1000268201_693b08cb0e',
       '1000268201_693b08cb0e', '1001773457_577c3a7d70',
       '1001773457_577c3a7d70', '1001773457_577c3a7d70',
       '1001773457_577c3a7d70', '1001773457_577c3a7d70',
       '1002674143_1b742ab4b8', '1002674143_1b742ab4b8',
       '1002674143_1b742ab4b8', '1002674143_1b742ab4b8',
       '1002674143_1b742ab4b8', '1003163366_44323f5815'], dtype='<U21')

In [ ]:
#output -> arry shape (16,225,225,3)
#stacked images 
#for loop to load each image store it in list and transform all list into arry

In [138]:
class DataPipeline:
    
    def __init__(self,df,batch_size, vocab_size, img_folder_path,model):
        self.df = df
        self.batch_size = batch_size
        self.vocab_size = vocab_size
        self.img_folder_path = img_folder_path
        self.prepare_dataset()
        self.encoder_model = model

    def prepare_dataset(self):


        # for loop to append X1,X2,y
        X1,X2,y = [],[],[]

        for idx, data in self.df.iterrows():

            seq = data["value_tokenized"]
            for i in range(1,len(seq)):
                X1.append(data["id"])
                X2.append(seq[0:i])
                y.append(seq[i])

        self.X1,self.X2,self.y = X1,X2,y
    
    def encode_all_images(self):
        
       
        
        l_toencode = np.unique(self.X1)
        dic_encoded={}
        for image_name in tqdm(l_toencode):
            img_path = self.img_folder_path+image_name + ".jpg"
            arr_path = self.img_folder_path+image_name + ".npy"
            if os.path.exist(img_path):
                img = image.load_img(img_path, target_size=(256,256,3))
                x = image.img_to_array(img)
                x = np.expand_dims(x, axis=0)
                arr= self.encoder_model.predict(x)[0]
                np.save(open(arr_path, 'wb'),arr)
        

        
        
    def load_images_encoded(self, imgs_to_load):
        
        
        l_toencode = np.unique(imgs_to_load)
        dic_encoded={}
        for image_name in l_toencode:
            arr_path = self.img_folder_path+image_name + ".npy"
            dic_encoded[image_name] = np.load(open(arr_path, 'rb'))
        
        features = []
        for image_name in imgs_to_load:
            features.append(dic_encoded[image_name])
            
        
        final_array = np.array(features)
        return final_array
        

    def seq_to_padded(self,seq_to_pad):
        inputs_seq_model = pad_sequences(seq_to_pad,padding='post',maxlen=36)
            
        return inputs_seq_model

    
    def to_cat(self, y_to_cat):
        # function to categorical
        y = tf.keras.utils.to_categorical(y_to_cat, num_classes=self.vocab_size+2)
        return y

    def __getitem__(self,idx):
        

        imgs_to_load = self.X1[idx * self.batch_size : (idx +1) * self.batch_size]
        x1_batch = self.load_images_encoded(imgs_to_load)
        
        seq_to_pad = self.X2[idx * self.batch_size : (idx +1) * self.batch_size]
        x2_batch = self.seq_to_padded(seq_to_pad)
        
        y_to_cat = self.y[idx * self.batch_size : (idx +1) * self.batch_size]
        y_batch = self.to_cat(y_to_cat)
        
        return ([x1_batch,
                x2_batch],
                y_batch)
                
    def __len__(self):
        return len(self.X1)// self.batch_size

In [113]:
dl = DataPipeline(df_all,16, 8765,)

TypeError: __init__() missing 1 required positional argument: 'img_folder_path'

In [114]:
dl.batch_size = 64

In [115]:
len(dl)

7450

In [116]:
type(dl)

__main__.DataPipeline

In [117]:
!pwd

/Users/ChrisKarg/code/CMaxK/robo_romeo/notebooks


In [139]:
 #run CNN model to encode X1
CNN_model = EfficientNetB0(
include_top=False, 
weights='imagenet', 
input_tensor=None,
input_shape= (256,256,3),
pooling=None)

In [140]:
test = DataPipeline(df_all,16,8765, "../raw_data/Flicker8k_Dataset/",CNN_model)

In [141]:
1346*5*33

222090

In [125]:
test.encode_all_images()

  0%|          | 0/8092 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '../raw_data/Flicker8k_Dataset/2258277193_586949ec62.jpg'

In [122]:
np.load(open('../raw_data/Flicker8k_Dataset/1000268201_693b08cb0e.npy','rb')).shape

(8, 8, 1280)

In [136]:
(X1,X2),y = test[1]

1
0:00:00.010115
0:00:00.000283
0:00:00.000446
